In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionResNetV2
from keras import regularizers
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [1]:
train_dir = "/kaggle/input/ck-dataset"

In [3]:
img_size = 96 #original size of the image
epochs = 20
batch_size = 96
lr=0.001
seed=32
print(seed)

32


In [ ]:
# Define train and validation data generators
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(img_size, img_size),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical",
    subset="training"
)
validation_generator = validation_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(img_size, img_size),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical",
    subset="validation"
)

Found 788 images belonging to 7 classes.
Found 193 images belonging to 7 classes.


In [ ]:
# Build CNN model
model = tf.keras.models.Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(img_size, img_size, 1)))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (5, 5), padding='same', activation='relu'))
model.add(Conv2D(512, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Conv2D(512, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(7, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compile the model with corrected learning rate argument
model.compile(optimizer=Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Display the model summary to ensure all layers are correct
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 96, 96, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 96, 96, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 96, 96, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 48, 48, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 48, 48, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 48, 48, 128)         │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 48, 48, 512)         │         590,336 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 48, 48, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 48, 48, 512)         │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 24, 24, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 24, 24, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 294912)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │      75,497,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 7)                   │           3,591 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 78,812,167 (300.64 MB)

 Trainable params: 78,809,479 (300.63 MB)

 Non-trainable params: 2,688 (10.50 KB)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Set up model checkpoint to save the best model based on validation accuracy
checkpoint = ModelCheckpoint('model_optimal.keras',
                             monitor='val_accuracy',
                             save_best_only=True,
                             verbose=1,
                             mode='auto',
                             save_weights_only=False)

In [ ]:
import os

# List files in the current working directory to check if the file is there
print(os.listdir())

['.config', 'drive', 'sample_data']


In [ ]:
# Adjust the path to save in the current working directory or specific folder
checkpoint = ModelCheckpoint('./model_optimal.keras',  # Add './' to specify current directory
                             monitor='val_accuracy',
                             save_best_only=True,
                             verbose=1,
                             mode='auto',
                             save_weights_only=False)

In [ ]:
# Train the model with checkpointing
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,g
    callbacks=[checkpoint]
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 62s/step - accuracy: 0.1559 - loss: 10.2512  
Epoch 1: val_accuracy improved from -inf to 0.19171, saving model to ./model_optimal.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 1046s 72s/step - accuracy: 0.1594 - loss: 10.2264 - val_accuracy: 0.1917 - val_loss: 8.7208
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 63s/step - accuracy: 0.3501 - loss: 8.9500  
Epoch 2: val_accuracy did not improve from 0.19171
13/13 ━━━━━━━━━━━━━━━━━━━━ 886s 67s/step - accuracy: 0.3482 - loss: 8.9362 - val_accuracy: 0.1244 - val_loss: 9.3551
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 63s/step - accuracy: 0.3495 - loss: 7.7017  
Epoch 3: val_accuracy did not improve from 0.19171
13/13 ━━━━━━━━━━━━━━━━━━━━ 883s 68s/step - accuracy: 0.3497 - loss: 7.6813 - val_accuracy: 0.0725 - val_loss: 10.7187
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 63s/step - accuracy: 0.4713 - loss: 6.3602  
Epoch 4: val_accuracy did not improve from 0.19171
13/13 ━━━━━━━━━━━━━━━━━━━━ 925s 68s/step - accuracy: 0.4709 - loss: 

In [6]:
from tensorflow.keras import applications, regularizers, Model
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import glorot_uniform

# Define the number of output classes (change according to your dataset)
classes = 10  # For example, if you have 10 classes

# Set seed and learning rate (you can modify these as per your requirements)
seed = 42
learning_rate = 0.001

# Define the base model using InceptionResNetV2 (excluding the top layer)
base_model = applications.InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(75, 75, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu', kernel_initializer=glorot_uniform(seed), kernel_regularizer=regularizers.L2(0.001), activity_regularizer=regularizers.L2(1e-4), bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu', kernel_initializer=glorot_uniform(seed), kernel_regularizer=regularizers.L2(0.001), activity_regularizer=regularizers.L2(1e-4), bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu', kernel_initializer=glorot_uniform(seed), kernel_regularizer=regularizers.L2(0.001), activity_regularizer=regularizers.L2(1e-4), bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# Output layer with softmax activation for multi-class classification
predictions = Dense(classes, activation='softmax', kernel_initializer='random_uniform', bias_initializer='zeros')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Set up the optimizer and loss function
optimizer = Adam(learning_rate=learning_rate)
loss = "categorical_crossentropy"

# Make all layers trainable
for layer in model.layers:
    layer.trainable = True

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

# Summary of the model to check the architecture
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 75, 75, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_203 (Conv2D)       │ (None, 37, 37, 32)     │            864 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_207   │ (None, 37, 37, 32)     │             96 │ conv2d_203[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_203            │ (None, 37, 37, 32)     │              0 │ batch_normalization_2… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_204 (Conv2D)       │ (None, 35, 35, 32)     │          9,216 │ activation_203[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_208   │ (None, 35, 35, 32)     │             96 │ conv2d_204[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_204            │ (None, 35, 35, 32)     │              0 │ batch_normalization_2… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_205 (Conv2D)       │ (None, 35, 35, 64)     │         18,432 │ activation_204[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_209   │ (None, 35, 35, 64)     │            192 │ conv2d_205[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_205            │ (None, 35, 35, 64)     │              0 │ batch_normalization_2… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_4           │ (None, 17, 17, 64)     │              0 │ activation_205[0][0]   │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_206 (Conv2D)       │ (None, 17, 17, 80)     │          5,120 │ max_pooling2d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_210   │ (None, 17, 17, 80)     │            240 │ conv2d_206[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_206            │ (None, 17, 17, 80)     │              0 │ batch_normalization_2… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_207 (Conv2D)  

 Total params: 55,298,922 (210.95 MB)

 Trainable params: 55,233,514 (210.70 MB)

 Non-trainable params: 65,408 (255.50 KB)

In [15]:
import os

# Create the dataset directory structure
os.makedirs('dataset/train/class_1', exist_ok=True)
os.makedirs('dataset/train/class_2', exist_ok=True)
os.makedirs('dataset/validation/class_1', exist_ok=True)
os.makedirs('dataset/validation/class_2', exist_ok=True)


In [17]:
train_generator = train_datagen.flow_from_directory(
    'dataset/train',  # Ensure this path is correct
    target_size=(75, 75),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'dataset/validation',  # Make sure to point to the validation directory
    target_size=(75, 75),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [18]:
# Prepare data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Train generator
train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(75, 75),
    batch_size=32,
    class_mode='categorical'
)

# Validation generator
validation_generator = validation_datagen.flow_from_directory(
    'dataset/validation',
    target_size=(75, 75),
    batch_size=32,
    class_mode='categorical'
)


Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
